# **Traveler Insights**

A medida que el turismo se vuelve cada vez más digital y las reseñas en línea influyen fuertemente en las decisiones de viaje, las plataformas digitales juegan un papel crucial en la comprensión de las preferencias y comportamientos de los turistas. A través de Traveler Insights, tu misión es desarrollar un modelo capaz de clasificar el sentimiento de estas reseñas en tres categorías principales: positivo, neutral o negativo. Esta tarea es clave para ayudar a la industria turística a comprender mejor las emociones de los visitantes, permitiéndoles adaptar sus ofertas para mejorar las experiencias y optimizar sus servicios.

Participantes:

Daniel Isaías Cruz González

Jaider Castilla Babilonia

# Librerías Python

---

In [32]:
# Importando librerias
%pip install pandas scikit-learn plotly
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


#from sklearn.model_selection import RandomizedSearchCV,RepeatedStratifiedKFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, confusion_matrix


from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.


# Contenido de datos

* **ID:** Un identificador numérico único para cada revisión.
* **Sitio:** La plataforma de la que procede la reseña (por ejemplo, Foursquare).
* **Índice del lugar:** Índice o identificador único del lugar reseñado en la plataforma.
* **Nombre del lugar:** Nombre del lugar turístico reseñado.
* **Enlace del lugar:** Enlace URL a la página del lugar turístico en la plataforma.
* **Municipio:** Ciudad o municipio donde se encuentra el lugar turístico.
* **Valoración:** La calificación global otorgada al lugar en forma de texto (por ejemplo, «8,9/10»).
* **Valoraciones:** El número total de valoraciones que ha recibido el lugar.
* **Precio:** Precio o coste de la visita al lugar (si está disponible).
* **Comentario:** El texto completo de la valoración.
* **Fecha:** Fecha de la valoración.
* **Votos a favor:** Número de upvotes que ha recibido la reseña.
* **Votos en contra:** Número de votos en contra que ha recibido la valoración.
* **Valoración_num:** Extracción numérica del campo Valoración (por ejemplo, 8,9 de «8,9/10»).
* **Variable objetivo Sentimiento:** Es la etiqueta de sentimiento que predecirá para los datos test.csv. Puede tomar uno de estos tres valores: positivo, neutro y negativo.

# Lectura y Análisis de los datos

---

In [33]:
# Carga del dataset "TRAIN"
data_train = pd.read_csv('dataset/train.csv')
data_train.head(5)

,ID,Sitio,Índice del lugar,Nombre del lugar,Enlace del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,380,Tripadvisor,50,Isla Múcura,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,4.5/5,281,Precio Desconocido,La isla es muy hermosa y tiene unas aguas muy ...,24 de agosto de 2017,0,0,4.5,neutral
1,196,Foursquare,30,Doki's comidas rápidas (Majagual),https://es.foursquare.com/v/dokis-comidas-r%C3...,Sincelejo,7.4/10,13 valoraciones,$,"Lahamburguesa de polloy elpatacónDoki, son del...","Abril 12, 2015",0,0,7.4,positivo
2,723,Tripadvisor,56,Playa la Coquerita,https://www.tripadvisor.co/Attraction_Review-g...,Coveñas,4.0/5,28,Precio Desconocido,"La playa es preciosa, de aguas mansas y playas...",14 de febrero de 2020,0,0,4.0,neutral
3,522,Tripadvisor,50,Isla Múcura,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,4.5/5,281,Precio Desconocido,es un paraíso de playa Para llegar a la isla l...,3 de octubre de 2012,0,0,4.5,neutral
4,1126,Tripadvisor,74,Covenas Diving Co.,https://www.tripadvisor.co/Attraction_Review-g...,Coveñas,5.0/5,259,Precio Desconocido,Es muy buena escuela de buceo; excelente los l...,4 de marzo de 2018,0,0,5.0,neutral


In [34]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                818 non-null    int64  
 1   Sitio             818 non-null    object 
 2   Índice del lugar  818 non-null    int64  
 3   Nombre del lugar  818 non-null    object 
 4   Enlace del lugar  817 non-null    object 
 5   Municipio         818 non-null    object 
 6   Valoración        818 non-null    object 
 7   Valoraciones      818 non-null    object 
 8   Precio            818 non-null    object 
 9   Comentario        818 non-null    object 
 10  Fecha             818 non-null    object 
 11  Votos a favor     818 non-null    int64  
 12  Votos en contra   818 non-null    int64  
 13  Valoración_num    818 non-null    float64
 14  Sentimiento       818 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory usage: 96.0+ KB


In [35]:
# Carga del dataset "TEST"
data_test = pd.read_csv('dataset/submission.csv')
data_test.head(5)

,ID,Sitio,Índice del lugar,Nombre del lugar,Enlace del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,980,Tripadvisor,74,Covenas Diving Co.,https://www.tripadvisor.co/Attraction_Review-g...,Coveñas,5.0/5,259,Precio Desconocido,"De las mejores experiencias de la vida, no se ...",25 de abril de 2021,0,0,5.0,NaN
1,1249,Tripadvisor,79,Anfibio Sports Adventure,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,5.0/5,34,Precio Desconocido,Me encanto el plan de Stand up paddle y de bic...,1 de julio de 2021,0,0,5.0,NaN
2,260,Foursquare,42,Playas De La Primera Ensenada,https://foursquare.com/v/playas-de-la-primera-...,Coveñas,6.9/10,9 valoraciones,Precio Desconocido,Excelentes playas para disfrutar del hermoso m...,"Abril 13, 2013",0,0,6.9,NaN
3,459,Tripadvisor,50,Isla Múcura,https://www.tripadvisor.co/Attraction_Review-g...,San Onofre,4.5/5,281,Precio Desconocido,"Experiencia inolvidable Es un paisaje exótico,...",7 de enero de 2015,0,0,4.5,NaN
4,309,Foursquare,48,Lile's Pizza,https://foursquare.com/v/liles-pizza/5185a7cb4...,Sincelejo,6.7/10,31 valoraciones,$,Excelente la caprichosa y los granizados ;),"Septiembre 19, 2013",0,0,6.7,NaN


In [36]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                818 non-null    int64  
 1   Sitio             818 non-null    object 
 2   Índice del lugar  818 non-null    int64  
 3   Nombre del lugar  818 non-null    object 
 4   Enlace del lugar  817 non-null    object 
 5   Municipio         818 non-null    object 
 6   Valoración        818 non-null    object 
 7   Valoraciones      818 non-null    object 
 8   Precio            818 non-null    object 
 9   Comentario        818 non-null    object 
 10  Fecha             818 non-null    object 
 11  Votos a favor     818 non-null    int64  
 12  Votos en contra   818 non-null    int64  
 13  Valoración_num    818 non-null    float64
 14  Sentimiento       818 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory usage: 96.0+ KB


#### ANÁLISIS DE NULOS

In [37]:
#total_nulos_train = 
print(f"número total de registros nulos en \"train\" : {data_train.isnull().sum().sum()}")
#total_nulos_test = 
print(f"número total de registros nulos en \"test\" : {data_test.isnull().sum().sum()}")
#nulos_por_columna = df.isnull().sum()
#nulos_por_columna

número total de registros nulos en "train" : 1
número total de registros nulos en "test" : 129


In [38]:
nulos_por_columna = data_train.isnull().sum()
nulos_por_columna

ID                  0
Sitio               0
Índice del lugar    0
Nombre del lugar    0
Enlace del lugar    1
Municipio           0
Valoración          0
Valoraciones        0
Precio              0
Comentario          0
Fecha               0
Votos a favor       0
Votos en contra     0
Valoración_num      0
Sentimiento         0
dtype: int64

In [39]:
nulos_por_columna = data_test.isnull().sum()
nulos_por_columna

ID                    0
Sitio                 0
Índice del lugar      0
Nombre del lugar      0
Enlace del lugar      0
Municipio             0
Valoración            0
Valoraciones          0
Precio                0
Comentario            0
Fecha                 0
Votos a favor         0
Votos en contra       0
Valoración_num        0
Sentimiento         129
dtype: int64

**Conclusión:** El único dataset que posee valores nulos es el de entrenamiento. Sin embargo, la característica dónde se encuentre dicho valor nulo ("Enlace del lugar") es prescindible para el análisis. Por lo que se procederá a eliminar dicha característica de ambos dataset.

In [40]:
data_train = data_train.drop("Enlace del lugar", axis=1)
data_test = data_test.drop("Enlace del lugar", axis=1)

In [41]:
data_train.head()

,ID,Sitio,Índice del lugar,Nombre del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,380,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,La isla es muy hermosa y tiene unas aguas muy ...,24 de agosto de 2017,0,0,4.5,neutral
1,196,Foursquare,30,Doki's comidas rápidas (Majagual),Sincelejo,7.4/10,13 valoraciones,$,"Lahamburguesa de polloy elpatacónDoki, son del...","Abril 12, 2015",0,0,7.4,positivo
2,723,Tripadvisor,56,Playa la Coquerita,Coveñas,4.0/5,28,Precio Desconocido,"La playa es preciosa, de aguas mansas y playas...",14 de febrero de 2020,0,0,4.0,neutral
3,522,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,es un paraíso de playa Para llegar a la isla l...,3 de octubre de 2012,0,0,4.5,neutral
4,1126,Tripadvisor,74,Covenas Diving Co.,Coveñas,5.0/5,259,Precio Desconocido,Es muy buena escuela de buceo; excelente los l...,4 de marzo de 2018,0,0,5.0,neutral


In [42]:
data_test.head()

,ID,Sitio,Índice del lugar,Nombre del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,980,Tripadvisor,74,Covenas Diving Co.,Coveñas,5.0/5,259,Precio Desconocido,"De las mejores experiencias de la vida, no se ...",25 de abril de 2021,0,0,5.0,NaN
1,1249,Tripadvisor,79,Anfibio Sports Adventure,San Onofre,5.0/5,34,Precio Desconocido,Me encanto el plan de Stand up paddle y de bic...,1 de julio de 2021,0,0,5.0,NaN
2,260,Foursquare,42,Playas De La Primera Ensenada,Coveñas,6.9/10,9 valoraciones,Precio Desconocido,Excelentes playas para disfrutar del hermoso m...,"Abril 13, 2013",0,0,6.9,NaN
3,459,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,"Experiencia inolvidable Es un paisaje exótico,...",7 de enero de 2015,0,0,4.5,NaN
4,309,Foursquare,48,Lile's Pizza,Sincelejo,6.7/10,31 valoraciones,$,Excelente la caprichosa y los granizados ;),"Septiembre 19, 2013",0,0,6.7,NaN


#### ANÁLISIS DE VALORES ÚNICOS

In [43]:
# Analizar las columnas y sus valores
for col in data_train.columns:
    unique_vals = data_train[col].nunique()
    print(f"Columna '{col}' tiene {unique_vals} valores únicos.")

Columna 'ID' tiene 818 valores únicos.
Columna 'Sitio' tiene 2 valores únicos.
Columna 'Índice del lugar' tiene 69 valores únicos.
Columna 'Nombre del lugar' tiene 73 valores únicos.
Columna 'Municipio' tiene 9 valores únicos.
Columna 'Valoración' tiene 23 valores únicos.
Columna 'Valoraciones' tiene 43 valores únicos.
Columna 'Precio' tiene 4 valores únicos.
Columna 'Comentario' tiene 809 valores únicos.
Columna 'Fecha' tiene 688 valores únicos.
Columna 'Votos a favor' tiene 16 valores únicos.
Columna 'Votos en contra' tiene 2 valores únicos.
Columna 'Valoración_num' tiene 23 valores únicos.
Columna 'Sentimiento' tiene 3 valores únicos.


**Conclusiones:** Tomando como punto de partida lo que se nos solicita en el reto, decidimos que las siguientes carácterísticas no son relevantes en el análisis de sentimientos a las reseñas:

* **ID** 
* **Sitio**
* **Indice del lugar**
* **Nombre del lugar**
* **Municipio**
* **Valoraciones**
* **Precio**
* **Fecha**
* **Valoración_num**
* ****

#### EDA VARIABLES CATEGÓRICAS

In [44]:
categorical_cols = data_train.select_dtypes(include=['object']).columns.tolist()

print("Columnas de las variables categóricas:")
for col in categorical_cols:
    print(" -",col)

Columnas de las variables categóricas:
 - Sitio
 - Nombre del lugar
 - Municipio
 - Valoración
 - Valoraciones
 - Precio
 - Comentario
 - Fecha
 - Sentimiento


**Conclusiones:**  Dada la naturaleza de algunas características, algunas no se consideran relevantes para el análisis. Por lo que se procederá a eliminar las siguientes carácterístcas:

* Sitio
* Nombre del luga
* Municip
* Valorac
* Valoraci
* Precioo
 
timiento

In [45]:
data_train = data_train.drop(columns=[ 'Sitio', 'Nombre del lugar', 'Municipio', 'Valoración', 'Valoraciones', 'Precio', 'Fecha'])
data_train.select_dtypes('O')

,Comentario,Sentimiento
0,La isla es muy hermosa y tiene unas aguas muy ...,neutral
1,"Lahamburguesa de polloy elpatacónDoki, son del...",positivo
2,"La playa es preciosa, de aguas mansas y playas...",neutral
3,es un paraíso de playa Para llegar a la isla l...,neutral
4,Es muy buena escuela de buceo; excelente los l...,neutral
...,...,...
813,Lomitoa lasfinas hierbas... buenisimo!,positivo
814,Hicimos un excelente fun dive con Stefania y G...,neutral
815,El parque a su alrededor estaba en remodelació...,neutral
816,"La mejor experiencia en Coveñas, totalmente re...",neutral


#### EDA VARIABLES NUMÉRICAS

In [46]:
def estadisticos_cont(num):
    #Calculamos describe
    estadisticos = num.describe().T
    #Añadimos la mediana
    estadisticos['median'] = num.median()
    #Reordenamos para que la mediana esté al lado de la media
    estadisticos = estadisticos.iloc[:,[0,1,8,2,3,4,5,6,7]]
    #Lo devolvemos
    return(estadisticos)

In [47]:
estadisticos_cont(data_train.select_dtypes('number'))

,count,mean,median,std,min,25%,50%,75%,max
ID,818.0,652.980440,660.5,373.715546,1.0,331.25,660.5,974.75,1298.0
Índice del lugar,818.0,52.808068,53.0,21.491306,1.0,50.00,53.0,74.00,81.0
Votos a favor,818.0,0.591687,0.0,1.586982,0.0,0.00,0.0,1.00,17.0
Votos en contra,818.0,0.001222,0.0,0.034964,0.0,0.00,0.0,0.00,1.0
Valoración_num,818.0,5.369438,5.0,1.463441,3.5,4.50,5.0,5.00,8.9


**Conclusión:** ID e Índice del lugar no aportan al entrénamiento. Se procederá a eleminar estas carácterísticas.

In [48]:
data_train = data_train.drop(columns=[ 'ID', 'Índice del lugar'])
data_train

,Comentario,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,La isla es muy hermosa y tiene unas aguas muy ...,0,0,4.5,neutral
1,"Lahamburguesa de polloy elpatacónDoki, son del...",0,0,7.4,positivo
2,"La playa es preciosa, de aguas mansas y playas...",0,0,4.0,neutral
3,es un paraíso de playa Para llegar a la isla l...,0,0,4.5,neutral
4,Es muy buena escuela de buceo; excelente los l...,0,0,5.0,neutral
...,...,...,...,...,...
813,Lomitoa lasfinas hierbas... buenisimo!,0,0,8.7,positivo
814,Hicimos un excelente fun dive con Stefania y G...,0,0,5.0,neutral
815,El parque a su alrededor estaba en remodelació...,0,0,4.0,neutral
816,"La mejor experiencia en Coveñas, totalmente re...",0,0,5.0,neutral


# Preprocesamiento: Entrenamiento y Normalización

In [49]:
# Convertir la columna Sentimiento a etiquetas numéricas
sentiment_mapping = {'positivo': 2, 'neutral': 1, 'negativo': 0}
data_train['Sentimiento_num'] = data_train['Sentimiento'].map(sentiment_mapping)

# Separamos las etiquetas
X_text_train = data_train['Comentario']
y_train = data_train['Sentimiento']

In [50]:
# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X_text_train, y_train, test_size=0.2, random_state=42)

# Lista de palabras de detención para español
spanish_stop_words = [
    "de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un", "para", 
    "con", "no", "una", "su", "al", "lo", "como", "más", "pero", "sus", "le", "ya", "o", 
    "este", "sí", "porque", "esta", "entre", "cuando", "muy", "sin", "sobre", "también", 
    "me", "hasta", "hay", "donde", "quien", "desde", "todo", "nos", "durante", "todos", 
    "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos", "e", "esto", 
    "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra", "él", 
    "tanto", "esa", "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco", 
    "ella", "estar", "estas", "algunas", "algo", "nosotros", "mi", "mis", "tú", "te", 
    "ti", "tu", "tus", "ellas", "nosotras", "vosotros", "vosotras", "os", "mío", "mía", 
    "míos", "mías", "tuyo", "tuya", "tuyos", "tuyas", "suyo", "suya", "suyos", "suyas", 
    "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", "vuestra", "vuestros", 
    "vuestras", "esos", "esas", "estoy", "estás", "está", "estamos", "estáis", "están", 
    "esté", "estés", "estemos", "estéis", "estén", "estaré", "estarás", "estará", 
    "estaremos", "estaréis", "estarán", "estaría", "estarías", "estaríamos", 
    "estaríais", "estarían", "estaba", "estabas", "estábamos", "estabais", "estaban", 
    "estuve", "estuviste", "estuvo", "estuvimos", "estuvisteis", "estuvieron", "estuviera", 
    "estuvieras", "estuviéramos", "estuvierais", "estuvieran", "estuviese", "estuvieses", 
    "estuviésemos", "estuvieseis", "estuviesen", "estando", "estado", "estados", 
    "estada", "estadas", "estad"
]

In [51]:
# Vectorizamos los comentarios usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(data_test['Comentario'])
print(data_train)

                                            Comentario  Votos a favor  \
0    La isla es muy hermosa y tiene unas aguas muy ...              0   
1    Lahamburguesa de polloy elpatacónDoki, son del...              0   
2    La playa es preciosa, de aguas mansas y playas...              0   
3    es un paraíso de playa Para llegar a la isla l...              0   
4    Es muy buena escuela de buceo; excelente los l...              0   
..                                                 ...            ...   
813             Lomitoa lasfinas hierbas... buenisimo!              0   
814  Hicimos un excelente fun dive con Stefania y G...              0   
815  El parque a su alrededor estaba en remodelació...              0   
816  La mejor experiencia en Coveñas, totalmente re...              0   
817  A mediados de enero aún conservaba la decoraci...              0   

     Votos en contra  Valoración_num Sentimiento  Sentimiento_num  
0                  0             4.5     neutral       

In [52]:
data_test.head()

,ID,Sitio,Índice del lugar,Nombre del lugar,Municipio,Valoración,Valoraciones,Precio,Comentario,Fecha,Votos a favor,Votos en contra,Valoración_num,Sentimiento
0,980,Tripadvisor,74,Covenas Diving Co.,Coveñas,5.0/5,259,Precio Desconocido,"De las mejores experiencias de la vida, no se ...",25 de abril de 2021,0,0,5.0,NaN
1,1249,Tripadvisor,79,Anfibio Sports Adventure,San Onofre,5.0/5,34,Precio Desconocido,Me encanto el plan de Stand up paddle y de bic...,1 de julio de 2021,0,0,5.0,NaN
2,260,Foursquare,42,Playas De La Primera Ensenada,Coveñas,6.9/10,9 valoraciones,Precio Desconocido,Excelentes playas para disfrutar del hermoso m...,"Abril 13, 2013",0,0,6.9,NaN
3,459,Tripadvisor,50,Isla Múcura,San Onofre,4.5/5,281,Precio Desconocido,"Experiencia inolvidable Es un paisaje exótico,...",7 de enero de 2015,0,0,4.5,NaN
4,309,Foursquare,48,Lile's Pizza,Sincelejo,6.7/10,31 valoraciones,$,Excelente la caprichosa y los granizados ;),"Septiembre 19, 2013",0,0,6.7,NaN


### Entrenamiento de los modelos

#### Modelo Random Forest Classifier

In [53]:
modelo = RandomForestClassifier(random_state=42, n_estimators=100)
modelo.fit(X_train_tfidf, y_train)
y_val_pred = modelo.predict(X_val_tfidf)
validation_report = classification_report(y_val, y_val_pred, target_names=['negativo', 'neutral', 'positivo'])
print(validation_report)

              precision    recall  f1-score   support

    negativo       0.00      0.00      0.00         4
     neutral       0.85      0.96      0.90       122
    positivo       0.78      0.55      0.65        38

    accuracy                           0.84       164
   macro avg       0.54      0.50      0.52       164
weighted avg       0.82      0.84      0.82       164



In [54]:
# Calculo de exactitud general
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Exactitud general en el conjunto de validación: {100*accuracy:.2f}%")

Exactitud general en el conjunto de validación: 84.15%


In [55]:
test_predictions = modelo.predict(X_test_tfidf)
print(test_predictions)

['neutral' 'neutral' 'neutral' 'neutral' 'positivo' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'positivo' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'positivo'
 'positivo' 'neutral' 'neutral' 'neutral' 'neutral' 'positivo' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'positivo' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'positivo' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'positivo' 'positivo' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'positivo' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutr

In [56]:
data_test['Sentimiento'] = test_predictions
data_test['Sentimiento'] = data_test['Sentimiento'].map({'negativo':0, 'neutral' :1, 'positivo' :2})
print(data_test['Sentimiento'])

0      1
1      1
2      1
3      1
4      2
      ..
124    1
125    1
126    2
127    1
128    1
Name: Sentimiento, Length: 129, dtype: int64


In [57]:
# Crear el archivo de envío
data_submission = data_test[['ID', 'Sentimiento']].head(129)
output_path = 'submission_data.csv'
data_submission.to_csv(output_path, index=False)
output_path

'submission_data.csv'